# TensorFlow逻辑回归处理MNIST数据集
本节基于回归学习对 MNIST 数据集进行处理，但将添加一些 TensorBoard 总结以便更好地理解 MNIST 数据集。  
MNIST由https://www.tensorflow.org/get_started/mnist/beginners提供。
大部分人已经对 MNIST 数据集很熟悉了，它是机器学习的基础，包含手写数字的图像及其标签来说明它是哪个数字。

对于逻辑回归，对输出 y 使用独热（one-hot）编码。因此，有 10 位表示输出，每位的值为 1 或 0，独热意味着对于每个图片的标签 y，10 位中仅有一位的值为 1，其余的为 0。

因此，对于手写数字 8 的图像，其编码值为 [0000000010]：
![](./Markdown_images/miao_17_TensorFlow_逻辑回归处理MNIST数据集/01.jpg)

# 具体做法

1. 导入所需的模块：

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

2. 可以从模块 input_data 给出的 TensorFlow 示例中获取 MNIST 的输入数据。该 one_hot 标志设置为真，以使用标签的 one_hot 编码。这产生了两个张量，大小为 [55000，784] 的 mnist.train.images 和大小为 [55000，10] 的 mnist.train.labels。mnist.train.images 的每项都是一个范围介于 0 到 1 的像素强度：

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../Database/MNIST_data/', one_hot=True) # 如指定的文件夹里没有数据集，则会自动下载

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../Database/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../Database/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../Database/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../Database/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train.images.shape

(55000, 784)

3. 在 TensorFlow 图中为训练数据集的输入 x 和标签 y 创建占位符：

In [4]:
# 因为每一个图像的大小为 28 * 28 像素个数为784，一次输入的图片数量未定义
x = tf.placeholder(tf.float32, [None, 784], name='X')
# 共有 10 个分类
y = tf.placeholder(tf.float32, [None, 10], name='Y')

4. 创建学习变量、权重和偏置：

In [5]:
w = tf.Variable(tf.zeros([784, 10]), name='W')
b = tf.Variable(tf.zeros([10]), name='B')

Instructions for updating:
Colocations handled automatically by placer.


5. 创建逻辑回归模型。TensorFlow OP 给出了 name_scope（"wx_b"）：

In [6]:
# 直接使用以上 with tf.name_scope('wx_b'): 生成一个在 tensorboard 中带可展开符号的一个域，并且支持嵌套操作
with tf.name_scope('wx_b') as scope:
    y_hot = tf.nn.softmax(tf.matmul(x, w) + b) # 没有使用激活函数

6. 训练时添加 summary 操作来收集数据。使用直方图以便看到权重和偏置随时间相对于彼此值的变化关系。可以通过 TensorBoard Histogtam 选项卡看到：

In [7]:
w_h = tf.summary.histogram('weights', w)
b_h = tf.summary.histogram('biases', b)

7. 定义交叉熵（cross-entropy）和损失（loss）函数，并添加 name scope 和 summary 以实现更好的可视化。
    tf.nn.softmax_cross_entropy_with_logits [使用说明](https://blog.csdn.net/yhily2008/article/details/80262321)
    使用 scalar summary 来获得随时间变化的损失函数。scalar summary 在 Events 选项卡下可见：

In [8]:
with tf.name_scope('cross-entropy') as scope:
    # labels 传入的概率，每个元素的之和相加要等于1 ，这里因为 y 为 one hot 编码
    # 也就是只有一个地方为1（或者说100%），其他地方为0（或者说0%）
    # 之所以用100%和0%描述，就是让它看起来像一个概率分布。（因为这是一个分类的模型）
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_hot))
    tf.summary.scalar('cross-entropy', loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



定义准确度

In [9]:
correct_prediction = tf.equal(tf.argmax(y_hot,1),tf.argmax(y,1)) # 对比预测值和实际值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

8. 采用 TensorFlow GradientDescentOptimizer，学习率为 0.01。为了更好地可视化，定义一个 name_scope：

In [10]:
learning_rate = 0.01
with tf.name_scope('Train') as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

9. 为变量进行初始化：

In [11]:
# Initializing the variables
init_op =tf.global_variables_initializer()

10. 组合所有的 summary 操作：

In [12]:
merged_summary_op = tf.summary.merge_all()

11. 现在，可以定义会话并将所有的 summary 存储在定义的文件夹中：

In [14]:
graph_path = 'graphs'
max_epochs = 30
batch_size = 100  # 每一个批次传入图片的数量
with tf.Session() as sess:
    sess.run(init_op)  # Initizlize all variables
    summary_writer = tf.summary.FileWriter('graphs',
                                           sess.graph)  # create an event file
    # Train
    for epoch in range(max_epochs):
        loss_avg = 0
        num_of_batch = int(mnist.train.num_examples / batch_size)
        for i in range(num_of_batch):
            # get the next batch of data
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, l, summary_str = sess.run([optimizer, loss, merged_summary_op],
                                         feed_dict={
                                             x: batch_xs,
                                             y: batch_ys
                                         })  # Run the optimizer
            loss_avg += l
            # add all summaries per batch
            summary_writer.add_summary(summary_str,epoch * num_of_batch + i)
        loss_avg = loss_avg / num_of_batch
        print('Epoch:{0} Loss:{1}'.format(epoch,loss_avg))
    print('Done')
    print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))

Epoch:0 Loss:2.2507057970220394
Epoch:1 Loss:2.091109765876423
Epoch:2 Loss:1.9603537728569724
Epoch:3 Loss:1.8824540281295776
Epoch:4 Loss:1.8336325773325834
Epoch:5 Loss:1.801699205311862
Epoch:6 Loss:1.7796908085996455
Epoch:7 Loss:1.7635018732331016
Epoch:8 Loss:1.7510123725370927
Epoch:9 Loss:1.7410572442141445
Epoch:10 Loss:1.7328677474368703
Epoch:11 Loss:1.7260154646093195
Epoch:12 Loss:1.720161413496191
Epoch:13 Loss:1.7150904581763529
Epoch:14 Loss:1.7106382562897422
Epoch:15 Loss:1.7066885933009062
Epoch:16 Loss:1.7031535905057733
Epoch:17 Loss:1.6999539572542364
Epoch:18 Loss:1.697056140682914
Epoch:19 Loss:1.6943885805390098
Epoch:20 Loss:1.691919253089211
Epoch:21 Loss:1.6896162139285695
Epoch:22 Loss:1.6874493382193825
Epoch:23 Loss:1.6853609186952765
Epoch:24 Loss:1.6832854383642024
Epoch:25 Loss:1.6810776485096324
Epoch:26 Loss:1.6783501215414567
Epoch:27 Loss:1.6739133204113354
Epoch:28 Loss:1.6670594794099982
Epoch:29 Loss:1.6611622366038237
Done
0.8614


12. 经过 30 个周期，准确率达到了 86.14%；经过 50 个周期，准确率达到了 89.36%；经过 100 个周期，准确率提高到了 90.91 %。

# 解读分析
这里使用张量 tensorboard--logdir=garphs 运行 TensorBoard。在浏览器中，导航到网址 localhost：6006 查看 TensorBoard。该模型图如下：
![](./Markdown_images/miao_17_TensorFlow_逻辑回归处理MNIST数据集/02.png)
在 Histogram 选项卡下，可以看到权重（weights）和偏置（biases）的直方图：
![](./Markdown_images/miao_17_TensorFlow_逻辑回归处理MNIST数据集/03.jpg)
![](./Markdown_images/miao_17_TensorFlow_逻辑回归处理MNIST数据集/04.jpg)
权重和偏置的分布如下：
![](./Markdown_images/miao_17_TensorFlow_逻辑回归处理MNIST数据集/05.jpg)
可以看到，随着时间的推移，偏置和权重都发生了变化。在该示例中，根据 TensorBoard 中的分布可知偏置变化的范围更大。在 Events 选项卡下，可以看到 scalar summary，即本示例中的交叉熵。下图显示交叉熵损失随时间不断减少：
![](./Markdown_images/miao_17_TensorFlow_逻辑回归处理MNIST数据集/06.jpg)